In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import numpy as np 
import pandas as pd
from keras.models import Model
from keras.layers import Dense, Embedding, Input,GRU
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text, sequence

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
max_features = 20000
maxlen = 100

In [ ]:
import pickle

#train = pickle.load(open('train.p','rb')).sample(10000)
#test = pickle.load(open('test.p','rb')).sample(10000)

In [ ]:
list_sentences_train = train["comment_text"].fillna("_NaN_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test.fillna("_NaN_").values

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
# train data
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
# test data
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:
import theano.ifelse
def cnn_rnn():
    embed_size = 256
    inp = Input(shape=(maxlen, ))
    main = Embedding(max_features, embed_size)(inp)
    main = Dropout(0.2)(main)
    main = Conv1D(filters=32, kernel_size=2, padding='same', activation='relu')(main)
    main = MaxPooling1D(pool_size=2)(main)
    main = Conv1D(filters=32, kernel_size=2, padding='same', activation='relu')(main)
    main = MaxPooling1D(pool_size=2)(main)
    main = GRU(32)(main)
    main = Dense(16, activation="relu")(main)
    main = Dense(6, activation="sigmoid")(main)
    model = Model(inputs=inp, outputs=main)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()     
    return model

In [ ]:
model = cnn_rnn()
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

print('Positive Labels ')
any_category_positive = np.sum(y,1)
print(pd.value_counts(any_category_positive))
X_t_train, X_t_test, y_train, y_test = train_test_split(X_t, y, 
                                                        test_size = 0.10, 
                                                        )
print('Training:', X_t_train.shape)
print('Testing:', X_t_test.shape)

In [ ]:
def batch_generator(X, y, batch_size, samplesN):
    number_of_batches = samplesN/batch_size
    counter=0
    shuffle_index = np.arange(np.shape(y)[0])
    np.random.shuffle(shuffle_index)
    X =  X[shuffle_index, :]
    y =  y[shuffle_index]
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[index_batch,:].todense()
        X_batch = np.expand_dims(X_batch, axis=2)
        y_batch = y[index_batch]
        counter += 1
        yield(np.array(X_batch),y_batch)
        if (counter < number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0

In [40]:
#sparse matrix input keras
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from keras import layers
from keras.layers import Dense, Embedding, Input,GRU, Flatten, AveragePooling1D,Bidirectional
from keras.layers import Conv2D,Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
import theano.ifelse
import numpy as np
train = pickle.load(open('train.p','rb'))
train=train[0:300]
batchSize=50
featuresN=50
model = Sequential()
print(model)

model.add(Conv1D(64, 4, activation='relu', padding='causal',input_shape=(featuresN,1)))
model.add(MaxPooling1D(4))
model.add(Bidirectional(GRU(60, return_sequences=True,name='lstm_layer',dropout=0.2,recurrent_dropout=0.2)))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 4, activation='relu', padding='causal'))
model.add(GlobalMaxPool1D())
model.add(layers.Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data


#submission_logreg = pd.read_csv('sample_submission.csv')
#submission_rf = pd.read_csv('sample_submission.csv')
tfidf = TfidfVectorizer(ngram_range=(1,2),max_features=featuresN)
features = tfidf.fit_transform(train['comment_text'])
print(features.shape)
#test = tfidf.transform(test)

#for col in ['toxic','severe_toxic','obscene','threat','insult','identity_hate']:
#    print(col)
#data=features.todense()
data=features
#data = np.expand_dims(data, axis=2)
X_train_sparse = sparse.csr_matrix(data)
print(X_train_sparse.shape[0])
labels=np.array(train['toxic'].tolist())#train[col].tolist()

model.fit_generator(generator=batch_generator(X_train_sparse, labels, batchSize, X_train_sparse.shape[0]),
                    nb_epoch=10, 
                    steps_per_epoch=(X_train_sparse.shape[0]/batchSize))
#model.fit(data, labels, epochs=10, batch_size=100)

#submission.to_csv('simple_LogRegClassWeight.csv', index=False)

#data = np.random.random((1000, 100))
#labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples


KeyboardInterrupt: 

In [41]:
print(labels.shape)

(300,)


In [33]:
from sklearn.metrics import roc_auc_score
X_train_sparse = sparse.csr_matrix(data)
#X_train_sparse = np.todense(X_train_sparse)[np.newaxis,:,:]
preds = model.predict_generator(generator=batch_generator(X_train_sparse, labels, batchSize, X_train_sparse.shape[0]),
                               steps=(X_train_sparse.shape[0]/batchSize))#np.expand_dims(X_train_sparse, axis=2))
print(roc_auc_score(train['toxic'], preds))



0.450473677899


In [36]:
labels.tolist().count(0)

269

In [37]:
labels.tolist().count(1)

31